In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import scipy.stats as stats
import requests
import rasterio
from rasterio.plot import show
import richdem as rd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.stats import poisson
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg
from rasterio.plot import show
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource


In [2]:
# Download a DEM 

# Your OpenTopography API key
OPENTOPO_API_KEY = "f95f74790401d94ee1989cd4cffea1d7"

# Define the bounding box for the area of interest
min_lon, min_lat = -111.5, 36.5
max_lon, max_lat = -102.5, 41.5

# OpenTopography API URL
url = "https://portal.opentopography.org/API/globaldem"

# Parameters for the API request
params = {
    'demtype': 'SRTMGL1',  # You can choose other DEM types like 'SRTMGL3', 'AW3D30', etc.
    'south': min_lat,
    'north': max_lat,
    'west': min_lon,
    'east': max_lon,
    'outputFormat': 'GTiff',
    'API_Key': OPENTOPO_API_KEY
}

# Make the API request
response = requests.get(url, params=params)

# Save the DEM file
dem_path = 'dem2.tif'
with open(dem_path, 'wb') as f:
    f.write(response.content)

with rasterio.open(dem_path) as src:
    # Calculate the window to read based on the desired coordinates
    window = rasterio.windows.from_bounds(min_lon, min_lat, max_lon, max_lat, transform=src.transform)
    dem = src.read(1, window=window)
    transform = src.window_transform(window)


In [3]:
dem = rd.LoadGDAL('dem2.tif')
dem

rdarray([[2694, 2698, 2703, ..., 1147, 1145, 1144],
         [2691, 2696, 2700, ..., 1148, 1146, 1144],
         [2689, 2694, 2698, ..., 1148, 1147, 1146],
         ...,
         [1938, 1937, 1936, ..., 1248, 1245, 1243],
         [1938, 1936, 1937, ..., 1246, 1245, 1243],
         [1938, 1939, 1941, ..., 1245, 1244, 1244]],
        shape=(18000, 32400), dtype=int16)

In [4]:
elev_data = []
row_arr = np.array_split(dem, 20)
#print(row_arr)
for arr in row_arr:
    col_arr = np.array_split(arr, 36, axis = 1)
    row_means = []
    for arr in col_arr:
        row_means.append(float(np.mean(arr)))
    row_means.reverse()
    elev_data.append(row_means)
    

elev_data

[[1164.745275308642,
  1259.4174086419753,
  1328.0675950617283,
  1392.8115925925927,
  1468.6310395061728,
  1547.4061913580247,
  1615.2626839506172,
  1683.9939679012346,
  1772.0719530864199,
  1903.2392802469135,
  2071.7216913580246,
  2398.0813592592594,
  2208.089612345679,
  2231.544825925926,
  2705.7709567901234,
  2978.505422222222,
  2356.085997530864,
  2209.4128814814817,
  2418.0644814814814,
  2332.1442469135804,
  2114.0441074074074,
  2042.5506382716048,
  2089.957451851852,
  2144.225335802469,
  2238.237797530864,
  2137.325750617284,
  2225.3240567901234,
  2044.6558851851853,
  1931.4317592592593,
  1978.1712691358025,
  2045.7489098765432,
  2051.013113580247,
  2131.9722,
  2218.269274074074,
  2023.609561728395,
  2349.166951851852],
 [1206.2687493827161,
  1270.677537037037,
  1328.4624185185185,
  1403.1874703703704,
  1481.8800703703703,
  1551.673013580247,
  1592.1038098765432,
  1672.4193827160493,
  1797.2968839506173,
  1913.35477654321,
  2130.261380

In [5]:
lon_edges = np.arange(-100.5, -111.5 - 0.25, -0.25)
lat_edges = np.arange(35.5, 42.5 + 0.25, 0.25)

lon_edges



array([-100.5 , -100.75, -101.  , -101.25, -101.5 , -101.75, -102.  ,
       -102.25, -102.5 , -102.75, -103.  , -103.25, -103.5 , -103.75,
       -104.  , -104.25, -104.5 , -104.75, -105.  , -105.25, -105.5 ,
       -105.75, -106.  , -106.25, -106.5 , -106.75, -107.  , -107.25,
       -107.5 , -107.75, -108.  , -108.25, -108.5 , -108.75, -109.  ,
       -109.25, -109.5 , -109.75, -110.  , -110.25, -110.5 , -110.75,
       -111.  , -111.25, -111.5 ])

In [6]:
lat_edges

array([35.5 , 35.75, 36.  , 36.25, 36.5 , 36.75, 37.  , 37.25, 37.5 ,
       37.75, 38.  , 38.25, 38.5 , 38.75, 39.  , 39.25, 39.5 , 39.75,
       40.  , 40.25, 40.5 , 40.75, 41.  , 41.25, 41.5 , 41.75, 42.  ,
       42.25, 42.5 ])

In [7]:
aval_2025 = pd.read_csv('../Data/every_reported_colorado_avalanche_2025.csv')
aval_2025['Date'] = aval_2025['Date'].str.split(' GMT').str[0]
aval_2025['Date'] = pd.to_datetime(aval_2025['Date'])
aval_2025['Date'] = aval_2025['Date'].dt.strftime('%b %d')
avalanche_date = aval_2025.loc[:, ["Date", "Longitude", "latitude"]]
avalanche_date = avalanche_date[avalanche_date['Date'] == 'Feb 25']
aval_2025 = aval_2025.loc[:, ['Longitude', 'latitude']]
for i in range(len(aval_2025)):
    if (aval_2025['Longitude'][i] == ' -'):
        aval_2025 = aval_2025.drop(i)
    elif ((float(aval_2025['Longitude'][i]) < -111.5 or float(aval_2025['Longitude'][i]) > -100.5) or (float(aval_2025['latitude'][i]) < 35.5 or float(aval_2025['latitude'][i]) > 42.5)):
        aval_2025 = aval_2025.drop(i)
aval_2025

,Longitude,latitude
0,-105.8925128,40.5208158
1,-105.7898892,39.8041827
2,-105.8211728,39.6442214
3,-105.8925128,40.5208158
4,-105.8925128,40.5208158
...,...,...
2793,-107.6064493,37.7908305
2794,-107.0376817,39.00537
2795,-107.0376817,39.00537
2796,-107.1211267,38.9072657


In [8]:
lon_index = np.where((-105.89 <= lon_edges[:-1]) & (-105.89 > lon_edges[1:]))[0]
lon_index
lat_index = np.where((40.52 >= lat_edges[:-1]) & (40.52 < lat_edges[1:]))[0]
lat_index
lon_index[0]

np.int64(21)

In [9]:
def find_surrounding_cells(i, j, max_i, max_j):
    surrounding = []
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            ni, nj = i + di, j + dj
            if 0 <= ni < max_i and 0 <= nj < max_j:
                surrounding.append((int(ni), int(nj)))
    return surrounding

find_surrounding_cells(42, 20, 45, 29)

[(41, 19),
 (41, 20),
 (41, 21),
 (42, 19),
 (42, 20),
 (42, 21),
 (43, 19),
 (43, 20),
 (43, 21)]

In [10]:
avalanche_cells = set()
for lon, lat in zip(aval_2025['Longitude'], aval_2025['latitude']):
    lon_index = np.where((float(lon) <= lon_edges[:-1]) & (float(lon) > lon_edges[1:]))[0]
    lat_index = np.where((float(lat) >= lat_edges[:-1]) & (float(lat) < lat_edges[1:]))[0]
    cells = find_surrounding_cells(int(lon_index[0]), int(lat_index[0]), 45, 29)
    for cell in cells:
        avalanche_cells.add(cell)

In [11]:
avalanche_cells = list(avalanche_cells)
avalanche_cells = sorted(avalanche_cells, key=lambda x: x[1])
avalanche_cells = sorted(avalanche_cells, key=lambda x: x[0])
avalanche_cells

[(16, 12),
 (16, 13),
 (16, 14),
 (17, 6),
 (17, 7),
 (17, 8),
 (17, 12),
 (17, 13),
 (17, 14),
 (18, 6),
 (18, 7),
 (18, 8),
 (18, 12),
 (18, 13),
 (18, 14),
 (19, 6),
 (19, 7),
 (19, 8),
 (19, 9),
 (19, 10),
 (19, 12),
 (19, 13),
 (19, 14),
 (19, 15),
 (19, 16),
 (19, 17),
 (19, 18),
 (19, 19),
 (19, 20),
 (20, 8),
 (20, 9),
 (20, 10),
 (20, 14),
 (20, 15),
 (20, 16),
 (20, 17),
 (20, 18),
 (20, 19),
 (20, 20),
 (20, 21),
 (21, 8),
 (21, 9),
 (21, 10),
 (21, 13),
 (21, 14),
 (21, 15),
 (21, 16),
 (21, 17),
 (21, 18),
 (21, 19),
 (21, 20),
 (21, 21),
 (22, 5),
 (22, 6),
 (22, 7),
 (22, 10),
 (22, 11),
 (22, 12),
 (22, 13),
 (22, 14),
 (22, 15),
 (22, 16),
 (22, 17),
 (22, 18),
 (22, 19),
 (22, 20),
 (22, 21),
 (23, 5),
 (23, 6),
 (23, 7),
 (23, 8),
 (23, 9),
 (23, 10),
 (23, 11),
 (23, 12),
 (23, 13),
 (23, 14),
 (23, 15),
 (23, 16),
 (23, 17),
 (23, 18),
 (23, 19),
 (23, 20),
 (23, 21),
 (23, 22),
 (24, 5),
 (24, 6),
 (24, 7),
 (24, 8),
 (24, 9),
 (24, 10),
 (24, 11),
 (24, 12),
 (24

In [12]:
precip_data = xr.open_dataset("../Data/Colorado_accum.nc")
precip_data
inst_data = xr.open_dataset("../Data/Colorado_instant.nc")
inst_data

<xarray.Dataset> Size: 11MB
Dimensions:     (valid_time: 672, latitude: 29, longitude: 45)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 5kB 2025-02-01 ... 2025-02-28T23:...
  * latitude    (latitude) float64 232B 42.5 42.25 42.0 ... 36.0 35.75 35.5
  * longitude   (longitude) float64 360B -111.5 -111.2 -111.0 ... -100.8 -100.5
    expver      (valid_time) <U4 11kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 4MB ...
    v10         (valid_time, latitude, longitude) float32 4MB ...
    t2m         (valid_time, latitude, longitude) float32 4MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-04-18T22:43 GRIB to CDM+CF via cfgrib-0.9.1...

In [13]:
avalanche_date

,Date,Longitude,latitude
680,Feb 25,-106.9269215,39.1680655
681,Feb 25,-107.7922035,37.908853
682,Feb 25,-107.8431967,37.8533975
683,Feb 25,-106.3889022,39.0226768
684,Feb 25,-105.724746,39.5993866
685,Feb 25,-106.1803007,39.4633203
686,Feb 25,-108.039188,37.449961
687,Feb 25,-106.5581472,39.0829085
688,Feb 25,-106.5581472,39.0829085
689,Feb 25,-105.8690439,39.8332199


In [14]:
elev = []
for lon, lat in avalanche_cells:
    elev.append(elev_data[lat-4][lon-8])
sum(elev)/len(elev)

2555.5532998971194

In [15]:
avalanche_count = []
precip_by_cell = []
rad_by_cell = []
temp_by_cell = []
u10_by_cell = []
v10_by_cell = []
elev_by_cell = []

for lon, lat in avalanche_cells:
    float_lon = [float(x) for x in avalanche_date['Longitude']]
    float_lat = [float(x) for x in avalanche_date['latitude']]
    count = len(avalanche_date[((float_lon <= lon_edges[lon]) & (float_lon >= lon_edges[lon+1])) &
                               ((float_lat >= lat_edges[lat]) & (float_lat <= lat_edges[lat+1]))])
    avalanche_count.append(count)
    precip_by_cell.append(precip_data['tp'].sel(valid_time='2025-02-25', latitude=lat_edges[lat], longitude=lon_edges[lon], method='nearest').item())
    rad_by_cell.append(precip_data['ssrd'].sel(valid_time='2025-02-25', latitude=lat_edges[lat], longitude=lon_edges[lon], method='nearest').item())
    temp_by_cell.append(inst_data['t2m'].sel(valid_time='2025-02-25', latitude=lat_edges[lat], longitude=lon_edges[lon], method='nearest').item())
    u10_by_cell.append(inst_data['u10'].sel(valid_time='2025-02-25', latitude=lat_edges[lat], longitude=lon_edges[lon], method='nearest').item())
    v10_by_cell.append(inst_data['v10'].sel(valid_time='2025-02-25', latitude=lat_edges[lat], longitude=lon_edges[lon], method='nearest').item())
    elev_by_cell.append(elev_data[lat-4][lon-8])

In [16]:
df = pd.DataFrame({'avalanche_count': avalanche_count, 'precip': precip_by_cell, 'rad': rad_by_cell, 'temp': temp_by_cell, 'u10': u10_by_cell, 'v10': v10_by_cell, 'elev': elev_by_cell})
df['rad'] /= 3600 
df['elev'] /= 1000 # Convert to km
#df['temp'] = (df['temp'] - 272.15) * 1.8 + 32 # Convert Kelvin to Fahrenheit
df

,avalanche_count,precip,rad,temp,u10,v10,elev
0,0,0.0,207.626667,291.231689,-2.314651,0.241074,1.997088
1,0,0.0,206.862222,288.930908,-3.240433,0.718613,2.193856
2,0,0.0,203.484444,288.354736,-2.436722,0.846542,1.985550
3,0,0.0,222.008889,286.587158,-1.464066,-0.930801,1.745888
4,0,0.0,215.804444,286.444580,-0.953323,-1.410294,1.560908
...,...,...,...,...,...,...,...
199,0,0.0,279.982222,278.009033,1.934372,-0.528458,1.607473
200,0,0.0,278.631111,278.671143,1.354294,0.327988,1.461432
201,0,0.0,281.831111,278.520752,1.671677,0.271347,1.683927
202,0,0.0,281.457778,277.862549,1.293747,0.338730,1.779874


In [17]:
10/204 # percentage of cells with an avalanche in it

0.049019607843137254

In [18]:
df['precip'].sum()

np.float64(7.62939453125e-06)

In [19]:
def model_parameters(reg, columns):
    """Returns a string with the linear regression model parameters for the given column names."""
    slopes = [f"{coef:.2f}({columns[i]})" for i, coef in enumerate(reg.coef_)]
    return " + ".join([f"{reg.intercept_:.2f}"] + slopes)

predictors = ['precip', 'rad', 'temp', 'u10', 'v10', 'elev']  
outcome = 'avalanche_count'

X = df[predictors]
y = df[outcome]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  # 80% training, 20% testing

model = LinearRegression().fit(X_train, y_train)
#model = LinearRegression().fit(df.loc[:, ['precip', 'rad', 'temp', 'u10', 'v10']], df['avalanche_count'])

print("Model:", model_parameters(model, X_train.columns))
print("Error:", root_mean_squared_error(y_test, model.predict(X_test)))

print("Model Coefficients:")
for feature, coef in zip(predictors, model.coef_):
    print(f"{feature:10s}: {coef:.4f}") 

Model: 5.76 + -150284.73(precip) + -0.00(rad) + -0.02(temp) + 0.00(u10) + -0.04(v10) + 0.00(elev)
Error: 0.18504609505509703
Model Coefficients:
precip    : -150284.7301
rad       : -0.0003
temp      : -0.0202
u10       : 0.0042
v10       : -0.0387
elev      : 0.0032


In [20]:
for predict in predictors:
    print("\n" + str(predict) + ": ")
    print(pg.corr(df[predict], df['avalanche_count']))
#pg.corr(df['precip'], df['avalanche_count'])


precip: 
           n         r          CI95%    p-val   BF10   power
pearson  204 -0.052499  [-0.19, 0.09]  0.45582  0.116  0.1158

rad: 
           n         r          CI95%     p-val   BF10     power
pearson  204  0.070927  [-0.07, 0.21]  0.313413  0.145  0.172314

temp: 
           n         r           CI95%     p-val  BF10     power
pearson  204 -0.164004  [-0.29, -0.03]  0.019081  1.34  0.652271

u10: 
           n         r          CI95%     p-val   BF10     power
pearson  204  0.127268  [-0.01, 0.26]  0.069684  0.449  0.443609

v10: 
           n         r          CI95%     p-val   BF10     power
pearson  204 -0.120717  [-0.25, 0.02]  0.085453  0.381  0.406608

elev: 
           n        r          CI95%     p-val   BF10     power
pearson  204  0.07406  [-0.06, 0.21]  0.292465  0.152  0.183714


In [21]:
df['avalanche_occurrence'] = (df['avalanche_count'] >= 1).astype(int)

logistic_model = sm.Logit(df['avalanche_occurrence'], df[['precip', 'rad', 'temp', 'u10', 'v10', 'elev']]).fit()

print("\nLogistic Regression Model Summary:")
print(logistic_model.summary())


         Current function value: 0.152969
         Iterations: 35

Logistic Regression Model Summary:
                            Logit Regression Results                            
Dep. Variable:     avalanche_occurrence   No. Observations:                  204
Model:                            Logit   Df Residuals:                      198
Method:                             MLE   Df Model:                            5
Date:                  Thu, 08 May 2025   Pseudo R-squ.:                  0.1540
Time:                          09:12:16   Log-Likelihood:                -31.206
converged:                        False   LL-Null:                       -36.887
Covariance Type:              nonrobust   LLR p-value:                   0.04466
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
precip       220.9626   3.91e+06   5.65e-05      1.000   -7.67e+06    7.67e+06
rad          

/opt/anaconda3/envs/DRP_env/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [31]:
coefficients = logistic_model.params
print(coefficients)
df_2 = pd.concat([coefficients, coefficients], axis=1, keys=['precip', 'rad', 'temp', 'u10', 'v10', 'elev'])
df_3 = pd.concat([df_2, coefficients], axis=1, keys=['precip', 'rad', 'temp', 'u10', 'v10', 'elev'])
df_3
# print(coefficients['precip'])
# print(coefficients['rad'])

precip    220.962552
rad         0.020479
temp       -0.050239
u10         0.846683
v10        -0.426985
elev        1.460859
dtype: float64


/var/folders/pr/qz3c0jc146355ryds4n93wjm0000gn/T/ipykernel_91304/118768625.py:3: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  df_2 = pd.concat([coefficients, coefficients], axis=1, keys=['precip', 'rad', 'temp', 'u10', 'v10', 'elev'])
/var/folders/pr/qz3c0jc146355ryds4n93wjm0000gn/T/ipykernel_91304/118768625.py:4: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  df_3 = pd.concat([df_2, coefficients], axis=1, keys=['precip', 'rad', 'temp', 'u10', 'v10', 'elev'])


precip                     rad
            precip         rad           0
precip  220.962552  220.962552  220.962552
rad       0.020479    0.020479    0.020479
temp     -0.050239   -0.050239   -0.050239
u10       0.846683    0.846683    0.846683
v10      -0.426985   -0.426985   -0.426985
elev      1.460859    1.460859    1.460859

In [22]:
poisson_model = smf.glm(formula='avalanche_count ~ precip + rad + temp + u10 + v10 + elev', data=df, family=sm.families.Poisson()).fit()
print("\nPoisson GLM Results:")
print(poisson_model.summary())



Poisson GLM Results:
                 Generalized Linear Model Regression Results                  
Dep. Variable:        avalanche_count   No. Observations:                  204
Model:                            GLM   Df Residuals:                      197
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -31.912
Date:                Thu, 08 May 2025   Deviance:                       44.596
Time:                        09:12:16   Pearson chi2:                     174.
No. Iterations:                    20   Pseudo R-squ. (CS):             0.1162
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    136.7014     56.0